In [5]:
# Import necessary initial libraries
import os
import sys
import importlib
from importlib import reload

In [2]:
# Sets the root directory of the project as the working directory
os.chdir('..')

In [3]:
# Check the current working directory
os.getcwd()

'/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms'

In [4]:
# Import necessary modules for this notebook
from libs.merge_geojson_grids import *

In [ ]:
# Reload the module to ensure the latest version is used
reload(sys.modules['libs.merge_geojson_grids'])

<module 'libs.merge_geojson_grids' from '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/libs/merge_geojson_grids.py'>

### Consolidação dos Arquivos GeoJSON das Grades Geoespaciais em um único arquivo

### 3. Métricas de Tempo

In [6]:
# Leitura do Arquivo Excel
file_name = 'tempo_processamento.xlsx'
df = pd.read_excel(file_name, sheet_name='Planilha1')

In [ ]:
# Exibe as primeiras linhas para validar
display(df.head(10))

,Lote 1: 0%| | 0/20 [00:00<?,?it/s]Ainda trabalhando...
0,Lote 1: 25%|██▌ | 5/20 [06:16<08:19,33.27s/it]Ainda trabalhando...
1,Lote 1: 50%|█████ | 10/20 [13:24<06:19,37.98s/it]Ainda trabalhando...
2,Ainda trabalhando...,NaN
3,Lote 1: 75%|███████▌ | 15/20 [21:42<03:43,44.72s/it]Ainda trabalhando...
4,Ainda trabalhando...,NaN
5,Lote 1: 100%|██████████| 20/20 [30:34<00:00,91.71s/it]
6,[SALVO] /content/drive/MyDrive/UFPR_PPGCG_DSc/...,NaN
7,[CONSOLIDADO] resultado_consolidado.geojson at...,NaN
8,Tempo do lote 1: 30.0 min 34.31 seg,NaN
9,Lote 2: 0%| | 0/20 [00:00<?,?it/s]Ainda trabalhando...


In [ ]:
# Exibe as últimas linhas para validar
display(df.tail(10))

,Lote 1: 0%| | 0/20 [00:00<?,?it/s]Ainda trabalhando...
1343,Lote 128: 35%|███▌ | 7/20 [11:04<19:10,88.51s/it] Ainda trabalhando...
1344,Lote 128: 50%|█████ | 10/20 [11:18<05:14,31.41s/it]Ainda trabalhando...
1345,Lote 128: 75%|███████▌ | 15/20 [17:08<02:59,35.94s/it]Ainda trabalhando...
1346,Lote 128: 100%|██████████| 20/20 [23:19<00:00,69.96s/it]
1347,[SALVO] /content/drive/MyDrive/UFPR_PPGCG_DSc/...,NaN
1348,[CONSOLIDADO] resultado_consolidado.geojson at...,NaN
1349,Tempo do lote 128: 23.0 min 20.09 seg,NaN
1350,Lote 129: 0%| | 0/20 [00:00<?,?it/s]Ainda trabalhando...
1351,Lote 129: 25%|██▌ | 5/20 [06:10<08:40,34.71s/it]Ainda trabalhando...
1352,Lote 129: 50%|█████ | 10/20 [12:29<05:58,35.85s/it]Ainda trabalhando...


In [8]:
# Função para extrair tempos dos lotes
def extrair_tempos(df):
    resultados = []

    for line in df.iloc[:, 0].dropna():
        match = re.search(r'Tempo do lote (\d+): (\d+\.?\d*) min (\d+\.?\d*) seg', line)
        if match:
            lote = int(match.group(1))
            minutos = float(match.group(2))
            segundos = float(match.group(3))
            total_segundos = int(minutos * 60 + segundos)
            resultados.append({
                'Lote': lote,
                'Minutos': minutos,
                'Segundos': segundos,
                'Total (s)': total_segundos,
                'Total (min)': total_segundos / 60,
                'Total (h)': total_segundos / 3600,
                'Total (dias)': total_segundos / (3600 * 24),
                'Duração (timedelta)': timedelta(seconds=total_segundos)
            })

    return pd.DataFrame(resultados)

In [9]:
# Aplicar a função para extrair os tempos
df_tempos = extrair_tempos(df)
df_tempos

,Lote,Minutos,Segundos,Total (s),Total (min),Total (h),Total (dias),Duração (timedelta)
0,1,30.0,34.31,1834,30.566667,0.509444,0.021227,0 days 00:30:34
1,2,31.0,23.44,1883,31.383333,0.523056,0.021794,0 days 00:31:23
2,3,27.0,41.91,1661,27.683333,0.461389,0.019225,0 days 00:27:41
3,4,28.0,20.47,1700,28.333333,0.472222,0.019676,0 days 00:28:20
4,5,25.0,27.84,1527,25.450000,0.424167,0.017674,0 days 00:25:27
...,...,...,...,...,...,...,...,...
123,124,25.0,5.30,1505,25.083333,0.418056,0.017419,0 days 00:25:05
124,125,24.0,26.13,1466,24.433333,0.407222,0.016968,0 days 00:24:26
125,126,27.0,30.72,1650,27.500000,0.458333,0.019097,0 days 00:27:30
126,127,28.0,9.41,1689,28.150000,0.469167,0.019549,0 days 00:28:09


In [10]:
# Cálculo do tempo total consolidado de todos os lotes
total_horas = df_tempos['Total (h)'].sum()
total_dias = df_tempos['Total (dias)'].sum()
print(f"Tempo Total (todos os lotes): {total_horas :.2f} horas")
print(f"Tempo Total (todos os lotes): {total_dias :.2f} dias")

Tempo Total (todos os lotes): 58.34 horas
Tempo Total (todos os lotes): 2.43 dias


In [11]:
# Gráfico de barras para visualizar os tempos de processamento
fig = px.bar(
    df_tempos,
    x='Lote',
    y='Total (min)',
    hover_data={
        'Minutos': ':.2f',
        'Segundos': ':.2f',
        'Total (h)': ':.3f',
        'Duração (timedelta)': True
    },
    labels={'Total (min)': 'Duração (minutos)'},
    title='Tempo de Processamento por Lote (Interativo)',
)

fig.update_traces(marker_color='royalblue')
fig.update_layout(
    xaxis_title='Lote',
    yaxis_title='Duração (minutos)',
    hoverlabel=dict(bgcolor="white", font_size=13)
)
fig.show()